In [19]:
#import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import plot_model

In [16]:
# Residual Block 

def residual_module(layer_in, n_filters):
  merge_input = layer_in
  # Check if the number of filters need to be increases, assumes channels last format
  if layer_in.shape[-1] != n_filters:
    layer_in = MaxPooling2D((2,2),strides=(2,2))(layer_in)
    merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
  # conv1
  conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
  #conv2
  conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
  # add filters, assumes filters/channels last
  layer_out = add([conv2, merge_input])
  layer_out = Activation('relu')(layer_out)
  return layer_out



In [20]:
#Define Moldel
#layer qty
layers_filters = [64,128,128,256,256,256,512,512]

# MODEL

# Input
visible = Input(shape=(256,256,3))
# Init Layers
layer = Conv2D(64, (7,7), strides=(2,2), kernel_initializer='he_normal')(visible)
layer = MaxPooling2D((3,3), strides=(2,2))(layer)

# Residual layers
for i in layers_filters:
  layer = residual_module(layer, i)

# Average Pool
#layer = AveragePooling2D((2,2), strides=(1,1), padding='same')(layer)
# Fully Connected
layer = Flatten()(layer)
layer = Dense(1000, activation='softmax')(layer)

# model
model = Model(inputs=visible, outputs=layer)

model.summary()
#plot model
#plot_model(model, show_shapes=True)

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 125, 125, 64) 9472        input_9[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 62, 62, 64)   0           conv2d_46[0][0]                  
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 62, 62, 64)   36928       max_pooling2d_8[0][0]            
_______________________________________________________________________________________